In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [86]:
trade_df = pd.read_csv('./data/Risk_Detection_Qualification/t_trade.csv', index_col='rowkey', dtype={'id': np.str})
trade_test_df = pd.read_csv('./data/Risk_Detection_Qualification/t_trade_test.csv', index_col='rowkey', dtype={'id': np.str})
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
login_df = pd.read_csv('./data/Risk_Detection_Qualification/t_login.csv', index_col='log_id', 
                       dtype={'id': np.str, 'timestamp': np.str},
                       parse_dates=['time'], date_parser=dateparse)
login_test_df = pd.read_csv('./data/Risk_Detection_Qualification/t_login_test.csv', index_col='log_id', 
                            dtype={'id': np.str, 'timestamp': np.str},
                            parse_dates=['time'], date_parser=dateparse)
merged_login_df = login_df.append(login_test_df)
login_df.shape, login_test_df.shape, merged_login_df.shape

((595645, 12), (87748, 12), (683393, 12))

In [91]:
# 得到一列不同的值
login_df['result'].unique()
login_df['id'].unique()
len(login_df['city'].unique()), len(login_df.index)
trade_df.count()

time       132719
id         132719
is_risk    132719
dtype: int64

显示某一列数值的分布情况

In [93]:
trade_df.shape, trade_df[trade_df['is_risk']==0].shape, trade_df[trade_df['is_risk']==1].shape
login_df.count()
#trade_df.is_risk.value_counts(normalize=True).plot(kind='bar', alpha=0.5)
trade_df.is_risk.value_counts(normalize=True)
#login_df.is_sec.value_counts(normalize=True).plot(kind='bar', alpha=0.5)
#login_test_df.is_sec.value_counts(normalize=True).plot(kind='bar', alpha=0.5)
#login_test_df.info()

0    0.972551
1    0.027449
Name: is_risk, dtype: float64

In [25]:
trade_df_by_id = trade_df['is_risk'].groupby(trade_df['id'])

id_risk_trade   = trade_df[trade_df['is_risk']==1]['id']
id_nonrisk_trade = trade_df[trade_df['is_risk']==0]['id']
checked_id = set(id_risk) & set(id_nonrisk)
len(id_risk), len(id_nonrisk), len(checked_id)

(3643, 129076, 455)

In [74]:
login_df_by_id = login_df.groupby(login_df['id']).count()
print(login_df_by_id)
#ratio = len(trade_df[trade_df['is_risk']==1])/len(trade_df)
#len(trade_df[trade_df['is_risk']==1]), len(trade_df)
# login_df['id']

        timelong  device  log_from   ip  city  result  timestamp  type  \
id                                                                       
100002         9       9         9    9     9       9          9     9   
100006        21      21        21   21    21      21         21    21   
100019         4       4         4    4     4       4          4     4   
100020        36      36        36   36    36      36         36    36   
100024        11      11        11   11    11      11         11    11   
100027         3       3         3    3     3       3          3     3   
100039        25      25        25   25    25      25         25    25   
100046        45      45        45   45    45      45         45    45   
100058        18      18        18   18    18      18         18    18   
100063        10      10        10   10    10      10         10    10   
100068         9       9         9    9     9       9          9     9   
100070         9       9         9    

显示trade_df和test_df中id的关系

In [89]:
login_id_set = set(login_df['id'])
login_test_id_set = set(login_test_df['id'])
trade_id_set = set(trade_df['id'])
trade_test_id_set = set(trade_test_df['id'])
set(merged_login_df['id']) == (trade_id_set | trade_test_id_set)
set(merged_login_df['id']) > trade_id_set, login_id_set > trade_id_set

(True, False)

对比显示：对于有危险登录的id和没有危险登录的id，在历史登录记录中ip数量 city数量分布

In [125]:
risk_id = trade_df[trade_df.is_risk==1]['id'].unique()
len(risk_id)
ip_count = login_df[['ip', 'id']].groupby('id').count()
risk_ip_count = ip_count[ip_count.index.isin(risk_id)]
# risk_ip_count['ip'].value_counts(normalize=True).plot(kind='bar', alpha=0.5)
nonrisk_ip_count = ip_count[~(ip_count.index.isin(risk_id))]
nonrisk_ip_count['ip'].mean()
#login_id_set

21.508036365031263

显示某一个数值是否有同一ip多个帐号的情况

In [132]:
id_per_ip_count = login_df[['ip', 'id']].groupby('ip').count()
id_per_ip_count['id'].max()
id_per_ip_count.sort_values(by='id', ascending = False)

,id
ip,
359080,20408
847186,1737
1128532,871
1551343,813
1297757,467
877190,455
621905,384
451870,379
743802,331


In [124]:
risk_ip_count
risk_ip_count['ip'].mean()

32.1841620626151